In [2]:
from google.colab.drive import mount
mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np

import nltk
import hazm
from hazm import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('/content/drive/My Drive/files/per.csv')

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي


In [ ]:
with open('/content/drive/My Drive/files/stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)

1495

In [ ]:
stemmer = hazm.Stemmer()

In [ ]:
dataset = pd.DataFrame(columns=('title_body', 'category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'title_body': ' '.join(title_body_tokenized_filtered_stemmed),
        'category': row['Category2'].replace('\n', '')
    }

In [ ]:
vectorizer = TfidfVectorizer()
le = LabelEncoder()

vectorizer.fit(dataset['title_body'])
X = vectorizer.transform(dataset['title_body'])

y = le.fit_transform(dataset['category'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
model = SVC(C=10 , kernel='linear' , probability=True , shrinking=True , random_state=1441)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       277
           1       0.64      0.68      0.66       245
           2       0.80      0.79      0.79       252
           3       0.91      0.89      0.90       262
           4       0.92      0.91      0.92       258
           5       0.79      0.69      0.74       245
           6       0.74      0.79      0.76       219
           7       0.85      0.82      0.83       255
           8       0.85      0.91      0.88       232
           9       0.92      0.94      0.93       262
          10       0.94      0.95      0.94       243

    accuracy                           0.84      2750
   macro avg       0.84      0.84      0.84      2750
weighted avg       0.84      0.84      0.84      2750

